<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Packages" data-toc-modified-id="Packages-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Packages</a></span></li><li><span><a href="#DairyDB" data-toc-modified-id="DairyDB-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>DairyDB</a></span></li></ul></div>

## Packages

In [4]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import re


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## DairyDB

In [124]:
from processing.hvr_extraction import get_all_hvr
from loading.main_sequence_loading import main_loading
from processing.hvr_statistics import main_stats

In [131]:
df_GG = main_loading('GreenGenes')

In [127]:
df_DB_DB = main_loading('DairyDB')

In [137]:
hvr_df_GG_DB = get_all_hvr(df_GG, primers_origin='DairyDB', save_name='hvr_GG_sequences_DB_primers')

In [129]:
hvr_df_DB_DB = get_all_hvr(df_DB_DB, primers_origin='DairyDB', save_name='hvr_DB_sequences_DB_primers')

In [138]:
hvr_df_GG_CH = get_all_hvr(df_GG, primers_origin='Chaudhary', save_name='hvr_GG_sequences_CH_primers')

MemoryError: 

In [134]:
hvr_df_DB_CH = get_all_hvr(df_DB_DB, primers_origin='Chaudhary', save_name='hvr_DB_sequences_CH_primers')

In [142]:
#main_stats(df_DB_DB, hvr_df_DB_DB, sequence_origin='DairyDB', primers_origin='DairyDB')
#main_stats(df_DB_DB, hvr_df_DB_CH, sequence_origin='DairyDB', primers_origin='Chaudhary')
#main_stats(df_GG, hvr_df_GG_DB, sequence_origin='GreenGenes', primers_origin='DairyDB')
#main_stats(df_GG, hvr_df_GG_CH, sequence_origin='GreenGenes', primers_origin='Chaudhary')

-------------------------------------------------- GLOBAL INFORMATION --------------------------------------------------
Total number of observations: 299499
Average sequence size: 1431.37
Standard deviation: 60.48
Minimal size: 1253
Maximal size: 2367
------------------------------------------------- TAROXOMY INFORMATION -------------------------------------------------
------------------------------------------------------- KINGDOM --------------------------------------------------------
Total number of unique kingdom: 2
Rank 1 in kingdom: BACTERIA with 25566 occurences
Rank 2 in kingdom: ARCHAEA with 831 occurences
273102 missing values
-------------------------------------------------------- PHYLUM --------------------------------------------------------
Total number of unique phylum: 68
Rank 1 in phylum: PROTEOBACTERIA with 8483 occurences
Rank 2 in phylum: FIRMICUTES with 7475 occurences
Rank 3 in phylum: BACTEROIDETES with 2564 occurences
Rank 4 in phylum: ACTINOBACTERIA with 21

Total number of remaining kingdom with at least one V3 found: 2 - 100.0% of total number of groups
Total number of remaining kingdom with at least one V4 found: 2 - 100.0% of total number of groups
Total number of remaining kingdom with at least one V5 found: 2 - 100.0% of total number of groups
Total number of remaining kingdom with at least one V6 found: 2 - 100.0% of total number of groups
Total number of remaining kingdom with at least one V7 found: 2 - 100.0% of total number of groups
Total number of remaining kingdom with at least one V8 found: 2 - 100.0% of total number of groups
Total number of remaining kingdom with at least one V9 found: 2 - 100.0% of total number of groups
Total number of remaining kingdom with at least one V12 found: 2 - 100.0% of total number of groups
Total number of remaining kingdom with at least one V23 found: 2 - 100.0% of total number of groups
Total number of remaining kingdom with at least one V34 found: 2 - 100.0% of total number of groups
Total n

Total number of remaining family with at least one V45 found: 406 - 97.0% of total number of groups
Total number of remaining family with at least one V56 found: 414 - 99.0% of total number of groups
Total number of remaining family with at least one V67 found: 410 - 98.0% of total number of groups
Total number of remaining family with at least one V78 found: 398 - 95.0% of total number of groups
Total number of remaining family with at least one V89 found: 303 - 72.0% of total number of groups
-------------------------------------------------------- GENUS ---------------------------------------------------------
Total number of unique genus: 823
286347 missing values - 95.61% of total
Total number of remaining genus with at least one V1 found: 499 - 61.0% of total number of groups
Total number of remaining genus with at least one V2 found: 733 - 89.0% of total number of groups
Total number of remaining genus with at least one V3 found: 788 - 96.0% of total number of groups
Total numbe